In [1]:
import numpy as np
import cv2
import mediapipe as mp
import pandas as pd
import os
import math

In [2]:
mp_face_mes = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
face_mesh = mp_face_mes.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.33, min_tracking_confidence=0.33)

In [3]:
images_open = os.listdir('mouth_open')
images_closed = os.listdir('mouth_closed')

print(len(images_open), len(images_closed))

320 277


In [4]:
def update_mesh_points(image):
    try:
        frame = image.copy()
    except AttributeError:
        return None
    H, W, _ = frame.shape
    results_mesh = face_mesh.process(frame)
    if results_mesh.multi_face_landmarks:
        mesh_points = np.array([np.multiply([p.x, p.y, p.z], [W, H, max(W, H)]).astype(int) for p in results_mesh.multi_face_landmarks[0].landmark])
        return mesh_points
    return None

In [5]:
MOUTH = [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95]

def calculate_mouth_area(mesh_points):
    ex1, ex2 = mesh_points[78], mesh_points[308]
    area = 0
    for i in range(len(MOUTH) - 1):
        area += (mesh_points[MOUTH[i], 0] * mesh_points[MOUTH[i + 1], 1]) - (mesh_points[MOUTH[i + 1], 0] * mesh_points[MOUTH[i], 1])
    area = abs(area / (2*(math.sqrt((ex1[0]-ex2[0])**2 + (ex1[1]-ex2[1])**2)**2)))
    return area

def calculate_width_over_height(mesh_points):
    ex1 = mesh_points[78]
    ex2 = mesh_points[308]
    ey1 = mesh_points[13]
    ey2 = mesh_points[14]
    try:
        reason = math.sqrt((ex1[0]-ex2[0])**2+(ex1[1]-ex2[1])**2+(ex1[2]-ex2[2])**2)/math.sqrt((ey1[0]-ey2[0])**2+(ey1[1]-ey2[1])**2+(ey1[2]-ey2[2])**2)
        if reason > 100: reason = 100
    except ZeroDivisionError:
        reason = 100
    return reason

def get_features(frame):
    mesh_points = update_mesh_points(frame)
    if mesh_points is None:
        return None
    area = calculate_mouth_area(mesh_points)
    reason = calculate_width_over_height(mesh_points)
    return (area, reason)

In [6]:
def resize_if_too_small(photo):
    if photo.shape[0] < 300 and photo.shape[1] < 300:
        photo = cv2.resize(photo, (300, 300), interpolation = cv2.INTER_CUBIC)
    return photo

In [7]:
results_open = []
results_closed = []

print('Processing open images...')
for image in images_open:
    frame = cv2.imread('mouth_open/'+image)
    if frame is None:
        continue
    frame = resize_if_too_small(frame)
    result = get_features(frame)
    if result:
        results_open.append((result[0], result[1]))

print('Processing closed images...')
for image in images_closed:
    frame = cv2.imread('mouth_closed/'+image)
    if frame is None:
        continue
    frame = resize_if_too_small(frame)
    result = get_features(frame)
    if result:
        results_closed.append((result[0], result[1]))

Processing open images...
Processing closed images...


In [8]:
print(len(results_open), len(results_closed))

244 193


In [9]:
results_open = np.array(results_open)
results_closed = np.array(results_closed)

In [10]:
df = pd.DataFrame(columns=['normalized_mouth_area', 'width_over_height', 'label'])

for value in results_open:
    df.loc[len(df)] = {'normalized_mouth_area': value[0], 'width_over_height': value[1], 'label': 'open'}
for value in results_closed:
    df.loc[len(df)] = {'normalized_mouth_area': value[0], 'width_over_height': value[1], 'label': 'closed'}

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 437 entries, 0 to 436
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   normalized_mouth_area  437 non-null    float64
 1   width_over_height      437 non-null    float64
 2   label                  437 non-null    object 
dtypes: float64(2), object(1)
memory usage: 13.7+ KB


In [11]:
df

,normalized_mouth_area,width_over_height,label
0,0.227025,2.506402,open
1,0.409836,100.000000,open
2,0.375000,100.000000,open
3,0.392562,100.000000,open
4,0.196721,4.135215,open
...,...,...,...
432,0.167047,24.643711,closed
433,0.162700,12.134661,closed
434,0.145841,15.357283,closed
435,0.194443,51.119957,closed


In [12]:
df.to_json('data.json', index=False)